### Libraries 

In [1]:
import torch
import pandas as pd
import time

### Ceasar Cipher Implementation

In [2]:
sample = 'Jim quickly realized that the beautiful gowns are expensive'
shift = 17
alphabet = 'ABCDEFGHIJKLMNOPQRSTVUWXYZabcdefghijklmnopqrstvuwxyz 0123456789,.!?:;’()-_—'
#0123456789 -!?.,:;'

In [3]:
def ceasar_cipher(text, shift, alphabet, decryption=False):
    output = ''
    if decryption == True:
        shift = -shift
    for x in range(len(text)):
        if text[x] in alphabet:
            output += alphabet[(alphabet.index(text[x]) + shift) % len(alphabet)]
        else:
            output += text[x] 
    return output

print('Original text:\n', sample)
enc_data = ceasar_cipher(sample, shift, alphabet)
print('Encrypted text:\n', enc_data)
dec_data = ceasar_cipher(enc_data, shift, alphabet, decryption=True)
print('Decrypted text:\n', dec_data)

Original text:
 Jim quickly realized that the beautiful gowns are expensive
Encrypted text:
 az2’6.zt01:’7ur1z;uv’9yr9’9yu’sur.9zw.1’x4!38’r7u’u?5u38z,u
Decrypted text:
 Jim quickly realized that the beautiful gowns are expensive


### Text for training

In [4]:
df = pd.read_csv('./Ceasar Salad.txt', delimiter="\t", header=None)
df.columns = ['original_sentence']
df.head()

,original_sentence
0,"Caesar salad, a simple salad made elegant, in ..."
1,"In 1924, during the Prohibition era (1920–33),..."
2,(1896–1956) moved his restaurant from San Dieg...
3,so that he could consume and serve alcohol wit...
4,"His restaurant, known as Caesar’s, lay just ac..."


In [5]:
df['encrypted_sentence'] = df['original_sentence'].apply(lambda x: ceasar_cipher(x, shift, alphabet))
df.head()

,original_sentence,encrypted_sentence
0,"Caesar salad, a simple salad made elegant, in ...",Tru8r7’8r1rvF’r’8z251u’8r1rv’2rvu’u1uxr39F’z3’...
1,"In 1924, during the Prohibition era (1920–33),...",Z3’)E-—F’v.7z3x’9yu’g74yzsz9z43’u7r’M)E-(–__NF...
2,(1896–1956) moved his restaurant from San Dieg...,"M)DEB–)EABN’24,uv’yz8’7u89r.7r39’w742’jr3’Vzux..."
3,so that he could consume and serve alcohol wit...,"84’9yr9’yu’t4.1v’t438.2u’r3v’8u7,u’r1t4y41’!z9..."
4,"His restaurant, known as Caesar’s, lay just ac...",Yz8’7u89r.7r39F’034!3’r8’Tru8r7L8F’1r:’ .89’rt...


### Preparation for training

In [6]:
sentence_corpus_orig = [[char for char in sentence] for sentence in df['original_sentence'].tolist()]
sentence_corpus_enc = [[char for char in sentence] for sentence in df['encrypted_sentence'].tolist()]
#if type(sentence) is str]
print(sentence_corpus_orig[0][:15])
print(sentence_corpus_enc[0][:15])

['C', 'a', 'e', 's', 'a', 'r', ' ', 's', 'a', 'l', 'a', 'd', ',', ' ', 'a']
['T', 'r', 'u', '8', 'r', '7', '’', '8', 'r', '1', 'r', 'v', 'F', '’', 'r']


In [7]:
#dict_char[None] = 0
dict_char = {char: i for i, char in enumerate(['None'] + [char for char in alphabet])}

In [8]:
def to_torch(text, sen_len, dictionary):
    X = torch.zeros((len(text), sen_len), dtype=int)
    for i in range(len(text)):
        for j, k in enumerate(text[i]):
            if j >= sen_len:
                break
            X[i,j] = dict_char.get(k, dict_char['None'])
    return X

In [9]:
X_enc = to_torch(sentence_corpus_enc, 100, dict_char)
X_enc.size()

torch.Size([50, 100])

In [10]:
X_orig = to_torch(sentence_corpus_orig, 100, dict_char)
#X_orig

In [11]:
def to_embeddings(X, dictionary):
    embeddings = torch.nn.Embedding(len(dict_char), 28)
    return embeddings(X)

In [12]:
X_orig_emb = to_embeddings(X_orig, dict_char)
#X_orig_emb

In [13]:
X_enc_emb = to_embeddings(X_enc, dict_char)
#X_enc_emb

### RNN 

In [14]:
class RNN(torch.nn.Module):
    def __init__(self):
        super(RNN, self).__init__()
        self.embed = torch.nn.Embedding(len(dict_char), len(dict_char))
        self.rnn = torch.nn.RNN(len(dict_char), 128, batch_first=True)
        self.linear = torch.nn.Linear(128, len(dict_char))
        
    def forward(self, sentences, state=None):
        embed = self.embed(sentences)
        o, a = self.rnn(embed)
        out = self.linear(o)
        return out

In [15]:
len(dict_char)

76

In [16]:
model = RNN()
criterion = torch.nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(model.parameters(), lr=.1)

In [17]:
X_orig.flatten().size()

torch.Size([5000])

In [18]:
X_enc[0]

tensor([20, 44, 48, 62, 44, 61, 70, 62, 44, 55, 44, 47,  6, 70, 44, 70, 62, 52,
        56, 59, 55, 48, 70, 62, 44, 55, 44, 47, 70, 56, 44, 47, 48, 70, 48, 55,
        48, 50, 44, 57, 63,  6, 70, 52, 57, 70, 52, 63, 62, 70, 46, 55, 44, 62,
        62, 52, 46, 70, 49, 58, 61, 56,  6, 70, 45, 68, 70, 63, 44, 45, 55, 48,
        62, 52, 47, 48, 70, 59, 61, 48, 59, 44, 61, 44, 63, 52, 58, 57,  7,  0,
         0,  0,  0,  0,  0,  0,  0,  0,  0,  0])

In [19]:
for ep in range(20):
    start = time.time()
    train_loss = 0.
    train_passed = 0

    for i in range(int(len(X_enc))):
        X_orig = X_orig.flatten()

        optimizer.zero_grad()
        answers = model.forward(X_enc)
        answers = answers.view(-1, len(dict_char))
        loss = criterion(answers, X_orig)
        train_loss += loss.item()

        loss.backward()
        optimizer.step()
        train_passed += 1
    print("Epoch {}. Time: {:.3f}, Train loss: {:.3f}".format(ep, time.time() - start, train_loss / train_passed))

Epoch 0. Time: 2.862, Train loss: 1.421
Epoch 1. Time: 2.022, Train loss: 0.409
Epoch 2. Time: 2.029, Train loss: 0.259
Epoch 3. Time: 2.111, Train loss: 0.194
Epoch 4. Time: 2.052, Train loss: 0.159
Epoch 5. Time: 2.099, Train loss: 0.136
Epoch 6. Time: 2.060, Train loss: 0.119
Epoch 7. Time: 2.020, Train loss: 0.106
Epoch 8. Time: 1.810, Train loss: 0.095
Epoch 9. Time: 1.626, Train loss: 0.086
Epoch 10. Time: 1.645, Train loss: 0.079
Epoch 11. Time: 1.876, Train loss: 0.072
Epoch 12. Time: 1.625, Train loss: 0.066
Epoch 13. Time: 1.587, Train loss: 0.061
Epoch 14. Time: 1.578, Train loss: 0.057
Epoch 15. Time: 1.587, Train loss: 0.053
Epoch 16. Time: 1.578, Train loss: 0.049
Epoch 17. Time: 1.601, Train loss: 0.046
Epoch 18. Time: 1.592, Train loss: 0.042
Epoch 19. Time: 1.577, Train loss: 0.040


In [20]:
def prediction(test):
    test_enc = [ceasar_cipher(test, 17, alphabet)]
    test_tensor = to_torch(test_enc, 100, dict_char)
    predict = model(test_tensor)
    predict = predict.squeeze(0)
    predict_test = ''
    for i, j in enumerate(predict):
        if j.argmax() != 0:
            predict_test += list(dict_char.keys())[list(dict_char.values()).index(j.argmax())]
    return print('Predicion:', predict_test)

In [21]:
prediction("Gaius Julius Caesar was a Roman general and statesman")

Predicion: gaius julius Caesar was a voman general and statesman
